In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from azure.storage.blob import BlobServiceClient
import os

bronze_path = f"{os.environ["AZURE_STORAGE_ACCOUNT_URL"]}/bronze"
silver_path = f"{os.environ["AZURE_STORAGE_ACCOUNT_URL"]}/silver"

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Transformar Bronze para Silver") \
    .getOrCreate()

# Listar as tabelas na camada Bronze (simulação de listar arquivos parquet)
bronze_tables = [f"{bronze_path}/{file}" for file in ["tabela1.parquet", "tabela2.parquet"]]  # Ajuste para seu ambiente

# Loop para processar cada tabela da camada Bronze
for table_path in bronze_tables:
    # Nome da tabela com base no caminho do arquivo
    table_name = table_path.split("/")[-1].replace(".parquet", "")

    # Leitura dos dados da camada Bronze
    df_bronze = spark.read.parquet(table_path)

    # Tratamentos básicos
    # Remover duplicatas
    # Preencher valores nulos com um padrão genérico
    # Preencher valores nulos em campos numéricos com zero
    df_silver = df_bronze \
        .dropDuplicates() \
        .na.fill("N/A") \
        .na.fill(0)

    # Normalização de tipos e colunas
    for column in df_silver.columns:
        # Exemplo de normalização: converter todas as strings para minúsculas
        if df_silver.schema[column].dataType.simpleString() == "string":
            df_silver = df_silver.withColumn(column, col(column).alias(column.lower()))
    
    # Tratamento adicional: Exemplo de transformação condicional
    df_silver = df_silver.withColumn(
        "status",
        when(col("status") == "ativo", "ativo")
        .when(col("status") == "inativo", "inativo")
        .otherwise("desconhecido")
    )

    # Caminho da tabela transformada na camada Silver
    silver_table_path = f"{silver_path}/{table_name}.parquet"

    # Gravação no Azure Data Lake em formato Parquet
    df_silver.write \
        .mode("overwrite") \
        .parquet(silver_table_path)

    print(f"Tabela {table_name} transformada e salva em {silver_table_path}")

# Encerrar sessão Spark
spark.stop()
